In [1]:
# Import packages
import requests
import pandas as pd
import shutil
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from bs4 import BeautifulSoup
from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment


print('packages loaded')

C:\Users\kp\AppData\Local\Temp\ipykernel_37188\1098827561.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


packages loaded


In [2]:
# Header to set the requests as a browser requests
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

print('headers loaded')

headers loaded


In [3]:

Test = 'https://careerservices.pace.edu/jobs/page/18/?ctag%5B0%5D=part-time&ctag%5B1%5D=internships&stag%5B0%5D=it-systems-support-cybersecurity&stag%5B1%5D=product-management-us-ui-research-design-technology-consulting&stag%5B2%5D=software-engineering-web-development-game-design&stag%5B3%5D=achieve-and-announce&stag%5B4%5D=connect-with-employers&stag%5B5%5D=grow-your-network&stag%5B6%5D=on-campus-employment&stag%5B7%5D=pace-funded-internships&stag%5B8%5D=search-for-internships-jobs&stag%5B9%5D=unleash-your-inner-professional&sort=date'

base_url = 'https://careerservices.pace.edu/jobs/page/{}'

start_page_number = 1
end_page_number = 100
ctags = ['part-time', 'internships','full-time']
stags = ['it-systems-support-cybersecurity', 'product-management-us-ui-research-design-technology-consulting', 'software-engineering-web-development-game-design', 'achieve-and-announce', 'connect-with-employers', 'grow-your-network', 'on-campus-employment', 'pace-funded-internships', 'search-for-internships-jobs', 'unleash-your-inner-professional']
sort_by = 'date'



print('query details loaded')

query details loaded


In [4]:
def convertJobsDataToArray(listOfJobs):
    data_dicts = []
    if listOfJobs:
        for job in listOfJobs:
            if job:
                # Elements
                titleElement = job.select_one("[rel='bookmark']")
                descElementWrapper = job.select_one("[class='entry-content']")
                companyElement = job.select_one("[class='company_name']")
                partOrFullTimeElement = job.select("[class='entry-meta-item']")
                # Variables
                jobTitle = ''
                jobUrl = ''
                jobDesc = ''
                companyName = ''
                partOrFullTime = ''
                # Logic
                if companyElement:
                    companyName = companyElement.text.strip()
                if titleElement:
                    jobTitle = titleElement.text.strip()
                    jobUrl = titleElement.get("href")
                if descElementWrapper:
                    paraGraphDescElement = descElementWrapper.find('p')
                    if paraGraphDescElement:
                        jobDesc = paraGraphDescElement.text.strip()
                if partOrFullTimeElement:
                    spanElementsListPartOrFullTime = partOrFullTimeElement[2]
                    if spanElementsListPartOrFullTime:
                        currentSpanElementsListPartOrFullTime = spanElementsListPartOrFullTime.find_all('span')
                        if currentSpanElementsListPartOrFullTime:
                            partOrFullTimeValue = currentSpanElementsListPartOrFullTime[1]
                            if partOrFullTimeValue:
                                partOrFullTime = partOrFullTimeValue.text.strip()
                if jobTitle:
                    defaultJobPath = 'https://careerservices.pace.edu/jobs/'
                    data_dict = {
                        'Title' : jobTitle,
                        'Company' : companyName,
                        'JobId' : jobUrl.split(defaultJobPath)[1],
                        'PartOrFullTime': partOrFullTime,
                        'url':jobUrl,
                        'Description' : jobDesc
                    }

                    # Add Dictionary in master empty List
                    data_dicts.append(data_dict)
                    
    return data_dicts
print('function loaded')   

function loaded


In [5]:
finalArrayList = []
for i in range(start_page_number, end_page_number):
        query_string = f'{base_url.format(i)}/?' +'&'.join([f'ctag%5B{i}%5D={tag}' for i, tag in enumerate(ctags)]) + '&'  + '&'.join([f'stag%5B{i}%5D={tag}' for i, tag in enumerate(stags)]) + f'&sort={sort_by}'
        
        response = requests.get(query_string, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        jobsWrapper = soup.select('div[id="featured-jobs-list"]')
        if jobsWrapper:
                currentJobsWrapperIndex = jobsWrapper[0]
                if currentJobsWrapperIndex:
                        listOfJobs = currentJobsWrapperIndex.select('.featured_job')
                        if listOfJobs:
                                jobList = convertJobsDataToArray(listOfJobs)
                                finalArrayList += jobList
        else:
                break
print('final details recieved loaded')

final details recieved loaded


In [6]:
df = pd.DataFrame(finalArrayList)

current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
filename = f'jobs_list.xlsx'
save_path = 'C:/Users/kp/Desktop/AmazonWebScraping-main/DailyJobHunt/' + filename

try:
    wb = load_workbook(filename)
except FileNotFoundError:
    # If the file doesn't exist, create a new workbook
    wb = Workbook()

if current_datetime in wb.sheetnames:
        print(f"Sheet with title '{current_datetime}' already exists in the Excel file.")
else:
        # Otherwise, create a new sheet with the current datetime
        ws = wb.create_sheet(title=current_datetime)
        headers = list(df.columns)
        ws.append(headers)
        for index, row in df.iterrows():
            ws.append(row.tolist())
        for row_index in range(1, len(headers) + 1):
            column_letter = get_column_letter(row_index)
            ws.column_dimensions[column_letter].width = max(len(str(header)) for header in headers)  
        for row_index in range(2, ws.max_row + 1):  
            ws.cell(row=row_index, column=3).hyperlink = ws.cell(row=row_index, column=5).value
        ws.auto_filter.ref = ws.dimensions
        wb.save(filename)
        shutil.move(filename, save_path)